In [1]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

- Bước đầu tiên để tính self-attention là tính ra các giá trị trung gian $ω$.
- $ω$ còn được gọi là attention scores (điểm chú ý).

In [2]:
query = inputs[1]   # x^2 (journey)
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)

print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
inputs.shape[0]

6

In [7]:
attn_scores_2.sum()

tensor(6.5617)

- `divide by sum`

In [8]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- `softmax naive`

In [11]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- Hàm `softmax` giúp đầu ra luôn là `số dương` và tổng bằng 1, vì xác suất không thể âm hoặc vượt quá 1.

- Tuy nhiên hàm `softmax_naive` đơn giản ở trên có thể `bị tràn số` (overflow) hoặc `mất chính xác` (underflow) khi đầu vào `quá lớn` hoặc `quá nhỏ`.
    + Hàm mũ $e^x$ tăng rất nhanh, $x = 700$ thì $e^x \approx 10^{304}$
    + Tương tự với khi $x$ âm lớn thì $e^x$ về 0 rất nhanh, $x = -700$ thì $e^x \approx 1^{-306}$

- Vì vậy, ta sẽ dùng `torch.softmax` được tối ưu cho các vấn đề tính toán này.

In [12]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- `context vector`

In [13]:
query = inputs[1]   # x^2 (journey)
context_vector_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vector_2 += attn_weights_2[i] * x_i
print("Context vector:", context_vector_2)

Context vector: tensor([0.4419, 0.6515, 0.5683])
